In [1]:
################################
# import all dependent libraries
################################
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import explode, split, col
from pyspark.sql.functions import udf, desc,asc,sum as Fsum
from pyspark.sql.types import (StringType, IntegerType, 
                               ArrayType,StructField, StructType)
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# agg() functions
# agg() functions
from pyspark.sql.functions import (count,first, sum as Fsum,
                                   last,countDistinct, sumDistinct)


########################################################################
# Set up a SparkSession in local mode to process datasets in a data lake
########################################################################
spark = SparkSession \
    .builder \
    .appName("ABS Census 2016 Data Analytics") \
    .config('spark.ui.port',3000) \
    .getOrCreate()
#spark.sparkContext.getConf().getAll()  # check sparksession full info


#############################################################################
#############################################################################
# Q1. Which postcode in Australia has the most diverse mix of countryOfBirth? 
#############################################################################
columns = StructType([
  StructField("abs_datasets", StringType(), True),
  StructField("POA", StringType(), True),
  StructField("birthCountry", StringType(), True),    
  StructField("wealthGroup", StringType(), True), 
  StructField("count", IntegerType(), True)    
])

Dff = spark.read\
    .schema(columns)\
    .option("mode","DROPMALFORMED")\
    .csv("WIDXbcPOA2016.csv")

Dff=Dff.withColumn("postcode",split(col("POA"),",").getItem(0))\
         .withColumn("state",split(col("POA"),",").getItem(1))\
         .drop("POA","abs_datasets").na.drop() #dropna()
Dff=Dff.filter(col("count") != 0)

DffQ1=Dff.dropDuplicates(["postcode","birthCountry"])
DffQ1=DffQ1.groupBy("postcode").count().orderBy(desc("count"))
DffQ1=DffQ1.withColumnRenamed("postcode","Australia Postcode")
DffQ1=DffQ1.withColumnRenamed("count","Count(Ethnic Groups) in 2016 Census")
DffQ1.show(n=10, truncate=False)



+------------------+-----------------------------------+
|Australia Postcode|Count(Ethnic Groups) in 2016 Census|
+------------------+-----------------------------------+
|3030              |98                                 |
|2155              |95                                 |
|2170              |89                                 |
|6065              |88                                 |
|2145              |86                                 |
|2010              |85                                 |
|2112              |85                                 |
|2153              |85                                 |
|2035              |84                                 |
|3029              |84                                 |
+------------------+-----------------------------------+
only showing top 10 rows

